In [2]:
# PSUDEO data
g = x -> cos(2x)+sin(x/2)  # λ function

X = range(1, stop=12, length=100)
X = Matrix(reshape(X, 1, :))

Y = map(g,X)

1×100 Matrix{Float64}:
 0.0632787  -0.0789055  -0.192898  …  -0.390548  -0.118139  0.144764

In [3]:
include("NTKernel.jl")

kernel (generic function with 1 method)

In [7]:
# nodes on hidden layer
Nh_list = [300] 
models = []
activation = sigmoid

InputDim = 1

for Nh in 1:length(Nh_list)
    push!(models,Chain(Dense(InputDim=>Nh_list[Nh],activation),Dense(Nh_list[Nh]=>InputDim))|>f64)
end

Kernels = []
for model in models
    push!(Kernels, kernel(model, X))
end

Kernels_Norms = []
for (i, K) in enumerate(Kernels)
    Nh = Nh_list[i] # Get the corresponding Nh for this kernel
    normalized_K = K / sqrt(Nh) # Normalize the kernel matrix
    push!(Kernels_Norms, normalized_K) # Store the normalized kernel
end

loss(a, b) = Flux.Losses.mse(models[1](a), b)
epoch = 1
for i = 1:epoch
    Flux.train!(loss, Flux.params(models[1]), [(X,Y)], Descent(0.01))
end

λ_values = []
for K in Kernels_Norms
    push!(λ_values, eigen(K).values) # Compute eigenvalues for each kernel and store
end

λ_values

1-element Vector{Any}:
 [-1.9539925233402755e-14, -1.8651746902253153e-14, -1.7763568394002505e-14, -1.3322676295501878e-14, -1.2434497875801753e-14, -1.1546319456101628e-14, -9.026052250943193e-15, -7.993605777301127e-15, -5.3547522210055705e-15, -5.329070518200751e-15  …  2.220446049250313e-14, 3.375077994860476e-14, 8.794579905448698e-14, 3.4492882658708833e-12, 2.904978880495183e-9, 3.929013231248316e-7, 3.261445494006039e-5, 0.033584158890642174, 9.189934502022489, 515.0940183261462]

In [41]:
using Plots

B = map_model(models[1],X)

Ŷ = []
for i in 1:length(X)
    push!(Ŷ, models[1](X[:, i])[1])  # Assuming models[1] outputs a 1-element array
end

scatter(X[:], Y[:], label="Real Data", xlabel="x", ylabel="cos(2x)+sin(x/2)")
scatter!(X[:], Ŷ[:], label="Predicted Data")

B==Ŷ

true

In [18]:
display(models[1])

Chain(
  Dense(1 => 300, σ),                   # 600 parameters
  Dense(300 => 1),                      # 301 parameters
)                   # Total: 4 arrays, 901 parameters, 7.289 KiB.